In [2]:
!pip3 install git+https://github.com/lovit/textrank.git

  Cloning https://github.com/lovit/textrank.git to c:\users\ativ\appdata\local\temp\pip-req-build-ugbfn1za
  Created wheel for textrank: filename=textrank-0.1.2-py3-none-any.whl size=11227 sha256=1c97954d1b84b3be035242a9115fab7ea35ba2cad46f58ba56b3f6d4f167d2a9
  Stored in directory: C:\Users\Ativ\AppData\Local\Temp\pip-ephem-wheel-cache-e1ulfmhi\wheels\92\77\3d\d596fd63962c68b352c2e39170c2cbe21b9fca35e89aafdf87
Successfully built textrank


You should consider upgrading via the 'c:\users\ativ\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [436]:
import pandas as pd
import numpy as np

df = pd.read_csv('review_included.csv')
df

,dining_id,total_name,rate,comment,source
0,0,카덴 연희본점,3.5,친절해요,네이버
1,0,카덴 연희본점,4.0,정갈하게 나와서 골고루 먹긴 좋았으나 사시미 종류가 좀 적은게 좀 아쉬웠어요 밥은 ...,네이버
2,0,카덴 연희본점,4.0,깔끔하고 여러가지를 즐길 수 있어서 좋았어요 ~,네이버
3,0,카덴 연희본점,4.0,특이하고 정성드린 런치정식이었어요~ 근데 고등어반찬은 비린맛이 좀 있더라구요 그래도...,네이버
4,0,카덴 연희본점,4.0,기대했던거와는 달리. 정식맛은 그닥 직원분들은 친절하세요,네이버
...,...,...,...,...,...
35572,477,디델리 이대점,5.0,최고예요,카카오
35573,477,디델리 이대점,0.0,학생으로 돌아간 느낌,카카오
35574,477,디델리 이대점,0.0,소고기치즈떡볶이,카카오
35575,477,디델리 이대점,5.0,사당동본점(없어진)에 가장 가까운 맛이고 디델리매니아로서 존재만으로 감사한 집! ...,카카오


In [437]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35577 entries, 0 to 35576
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dining_id   35577 non-null  int64  
 1   total_name  35577 non-null  object 
 2   rate        32147 non-null  float64
 3   comment     35516 non-null  object 
 4   source      35577 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.4+ MB


In [438]:
df['comment'] = df['comment'].str.replace(pat=r'[^ㄱ-ㅣ가-힣]', repl=r' ', regex=True)
df['comment'] = df['comment'].str.strip()
df['comment'].replace('', np.nan, inplace=True)
df.dropna(subset=['comment'], inplace=True)

In [439]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35118 entries, 0 to 35576
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dining_id   35118 non-null  int64  
 1   total_name  35118 non-null  object 
 2   rate        31695 non-null  float64
 3   comment     35118 non-null  object 
 4   source      35118 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.6+ MB


In [440]:
df.reset_index(drop=True, inplace=True)

In [441]:
pre_dining_keyword = pd.DataFrame(columns=['dining_id', 'keywords'])

In [442]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w)]
    return words

In [443]:
from textrank import KeywordSummarizer

for i in range(478):
    tmp = df[df['dining_id'] == i]
    if len(tmp) > 2:
        summarizer = KeywordSummarizer(tokenize=komoran_tokenizer)
        keywords = summarizer.summarize(tmp.comment.values, topk=10)
        words = []
        for word, rank in keywords:
            word = word[:word.index('/')]
            words.append(word)
        pre_dining_keyword.loc[i] = [i, words]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [444]:
pre_dining_keyword

,dining_id,keywords
0,0,"[우동, 면, 카, 데, 카레, 맛, 튀김, 것, 냉, 발]"
1,1,"[커피, 층, 분위기, 사람, 곳, 때, 공간, 음악, 점, 것]"
2,2,"[예약, 맛, 동파육, 음식, 탕수육, 것, 짬뽕, 새우, 거, 이연복]"
3,3,"[사장, 맛, 커피, 자리, 분위기, 카페, 곳, 커피, 직원, 것]"
4,4,"[신촌, 연경, 맛집, 그램, 맛, 탕수육, 짬뽕, 스타, 이대, 중국집]"
...,...,...
473,473,"[맛집, 신촌, 카, 우, 히우, 맛, 팟타이, 태국, 메뉴, 때]"
474,474,"[갈비, 서서, 남, 신촌, 식당, 서, 맛, 집, 먹, 연]"
475,475,"[신촌, 비아, 메렝게, 맛집, 맛, 멕시칸, 타코, 엔칠라다, 멕시코, 추천]"
476,476,"[카페, 신촌, 포티, 드, 포터, 해리, 쿠키, 데이트, 맛, 랄]"


In [463]:
pre_dining_keyword.to_csv('pre_dining_keyword.csv', index=False, encoding='utf-8-sig')

In [445]:
keywords = set()

In [449]:
for idx, row in pre_dining_keyword.iterrows():
    keywords.update(row['keywords'])

In [450]:
keywords

{0,
 '베네딕트',
 '꼬치',
 '남도',
 '종',
 '참치',
 '도넛',
 '할',
 '박',
 '삼색',
 '고추',
 '스무디',
 '커피',
 '패티',
 '것',
 '맛집',
 '절대',
 '스토리',
 '이벤트',
 '오징어',
 '탐앤탐스',
 '둑',
 '라멘',
 '비밀',
 '길',
 '목살',
 '동태',
 '손님',
 '푼',
 '폭',
 '레',
 '유부',
 '얼굴',
 '포크',
 '소',
 '방콕',
 '네일',
 '족발',
 '소고기',
 '전어',
 '마스크',
 '요네',
 '재료',
 '단자',
 '오랜만',
 '깐',
 '시식',
 '적',
 '듯',
 '버거킹',
 '도토리',
 '섬',
 '위',
 '정말',
 '인분',
 '양초',
 '케이크',
 '제임스',
 '사',
 '음료',
 '매운탕',
 '빵집',
 '서대문구',
 '샤브샤브',
 '양도',
 '고등어',
 '와이파이',
 '육회',
 '초밥',
 '버거',
 '환영',
 '오레오',
 '콩나물',
 '시간',
 '익스프레스',
 '새우',
 '고르드',
 '야키토리',
 '냉면',
 '단맛',
 '이스',
 '예약',
 '메가',
 '샐러드',
 '중',
 '앤',
 '탕수육',
 '기묘',
 '사람',
 '개인',
 '이집',
 '감바스',
 '번',
 '롤링',
 '감자전',
 '치아바타',
 '코코넛',
 '비',
 '아줌마',
 '공복',
 '두리',
 '삼',
 '폴',
 '계란',
 '음식',
 '사이',
 '으',
 '표기',
 '뉴타운',
 '살',
 '선택',
 '러',
 '센',
 '연희동',
 '떡',
 '슈가',
 '이랑',
 '서울',
 '도미노피자',
 '킹',
 '언니',
 '정량',
 '발리',
 '밀크티',
 '칠리',
 '떡볶이',
 '조',
 '링',
 '세팅',
 '밭',
 '다이',
 '미가',
 '포차',
 '버터',
 '마들렌',
 '드레싱',
 '활',
 '른당',
 '메',
 '시카고피자',
 '슈크림

In [454]:
keywords.remove(0)

In [460]:
keywords = list(keywords)

In [461]:
keyword = pd.DataFrame({'id': range(0, len(keywords)), 'word': keywords})

In [462]:
keyword

,id,word
0,0,베네딕트
1,1,꼬치
2,2,남도
3,3,종
4,4,참치
...,...,...
1091,1091,편리
1092,1092,안주
1093,1093,녹
1094,1094,오뎅


In [464]:
keyword.to_csv('keyword.csv', index=False, encoding='utf-8-sig')

In [517]:
dining_keyword = pd.DataFrame(columns=['dining_id', 'keyword_id'])

In [518]:
for idx, row in pre_dining_keyword.iterrows():
    for word in row['keywords']:
        tmp = keyword[keyword['word'] == word]
        dining_keyword = dining_keyword.append({'dining_id': idx, 'keyword_id': tmp.id.iloc[0]}, ignore_index=True)

In [519]:
dining_keyword

,dining_id,keyword_id
0,0,587
1,0,275
2,0,821
3,0,237
4,0,870
...,...,...
4633,477,143
4634,477,31
4635,477,733
4636,477,543


In [521]:
dining_keyword.to_csv('dining_keyword.csv', index=False, encoding='utf-8-sig')